In [227]:
import pandas as pd
df = pd.read_csv(r"C:\Users\HP\Documents\DATASCIENCE\nlp-getting-started\train.csv")
dftest = pd.read_csv(r"C:\Users\HP\Documents\DATASCIENCE\nlp-getting-started\test.csv")

In [229]:
train = df[:7000]
validation = df[7000:]

In [235]:
train

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
6995,10031,twister,NaN,@ellenfromnowon 7-speed nexus shifter å£9! (Fo...,0
6996,10032,twister,"Calgary, Alberta",Anyone wanna come over and watch Twister with ...,0
6997,10034,twister,NY,Crazy Mom Threw Teen Daughter a NUDE Twister S...,1
6998,10035,twister,"Plano,TX",@briannafrost Twister with Bill Paxton and Hel...,0


In [236]:
validation

,id,keyword,location,text,target
7000,10037,twister,"Midwest City, OK",happy 24 twister thank you for all the laughs ...,0
7001,10038,twister,NaN,how many men would a human hew if a human coul...,0
7002,10039,twister,NaN,i liked a video extreme paint twister,0
7003,10040,twister,NaN,tornadoes el nino may give canada s twister se...,1
7004,10041,twister,"Lisbon, Portugal",like gg was bad in the end but at least i care...,0
...,...,...,...,...,...
7608,10869,bridge,NaN,two giant cranes holding a bridge collapse int...,1
7609,10870,wild,NaN,ahrary the out of control wild fires in califo...,1
7610,10871,volcano,NaN,m1 94 01 04 utc 5km s of volcano hawaii,1
7611,10872,police,NaN,police investigating after an e bike collided ...,1


In [237]:
dftest

,id,keyword,location,text
0,0,crash,NaN,just happened a terrible car crash
1,2,earthquake,NaN,heard about earthquake is different cities sta...
2,3,forest,NaN,there is a forest fire at spot pond geese are ...
3,9,apocalypse,NaN,apocalypse lighting spokane wildfires
4,11,typhoon,NaN,typhoon soudelor kills 28 in china and taiwan
...,...,...,...,...
3258,10861,earthquake,NaN,earthquake safety los angeles safety fasteners...
3259,10865,storm,NaN,storm in ri worse than last hurricane my city ...
3260,10868,derailment,NaN,green line derailment in chicago
3261,10874,hazardous,NaN,meg issues hazardous weather outlook hwo


In [230]:
class preprocess():
    def __init__(self,bigdataorig,train,test = False, validation = False):
        self.train = train
        self.test = test
        self.validation = validation
        self.bigdataorig = bigdataorig
        
    def transform(self):
        if self.test == False and self.validation == False:
            zero_train = self.train[self.train['target'] == 0]
            one_train = self.train[self.train['target'] == 1]
            for i in zero_train.index:
                if len(zero_train) != len(one_train):
                    zero_train = zero_train.drop(i,axis = 0)
            dataset = pd.concat([one_train,zero_train])
            
            import re
            u = []
            for i in dataset['text']:
                cleaned = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",i).split())
                cleaned= cleaned.lower()
                u.append(cleaned)
            dataset['text'] = u
            listof = list(self.bigdataorig.keyword.unique())[1:]
            v= []
            for i in [i for i in listof if '%20' in i]:
                v.append((i.split('%20')))

            for i in v:
                for u in i:
                    listof.append(u)   
            a = []
            for k in dataset.index:
                if [i for i in dataset['text'][k].split() if i in listof]:
                    a.append([i for i in dataset['text'][k].split() if i in listof][0])
                else:
                    a.append('fire')
            f = []
            c = 0
            for i in (dataset['keyword'].index):
                if type(dataset['keyword'][i]) != str:
                    f.append(a[c])
                else:
                    f.append(dataset['keyword'][i])
                c+=1
            dataset['keyword'] = f
            dataset = dataset.drop(['location'],axis = 1)
            return dataset
        
        if self.test == False and self.validation == True:
            import re
            u = []
            for i in self.train['text']:
                cleaned = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",i).split())
                cleaned= cleaned.lower()
                u.append(cleaned)
            self.train['text'] = u
            listof = list(self.bigdataorig.keyword.unique())[1:]
            v= []
            for i in [i for i in listof if '%20' in i]:
                v.append((i.split('%20')))

            for i in v:
                for u in i:
                    listof.append(u)   
            a = []
            for k in self.train.index:
                if [i for i in self.train['text'][k].split() if i in listof]:
                    a.append([i for i in self.train['text'][k].split() if i in listof][0])
                else:
                    a.append('fire')
            f = []
            c =0
            for i in self.train['keyword'].index:
                if type(self.train['keyword'][i]) != str:
                    f.append(a[c])
                else:
                    f.append(self.train['keyword'][i])
                c+=1
            self.train['keyword'] = f
            self.train = self.train.drop(['location'],axis = 1)
            return self.train
        
        if self.test == True and self.validation == False:
            import re
            u = []
            for i in self.train['text']:
                cleaned = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",i).split())
                cleaned= cleaned.lower()
                u.append(cleaned)
            self.train['text'] = u
            listof = list(self.bigdataorig.keyword.unique())[1:]
            v= []
            for i in [i for i in listof if '%20' in i]:
                v.append((i.split('%20')))

            for i in v:
                for u in i:
                    listof.append(u)   
            a = []
            for k in self.train.index:
                if [i for i in self.train['text'][k].split() if i in listof]:
                    a.append([i for i in self.train['text'][k].split() if i in listof][0])
                else:
                    a.append('fire')
            f = []
            c =0
            for i in self.train['keyword'].index:
                if type(self.train['keyword'][i]) != str:
                    f.append(a[c])
                else:
                    f.append(self.train['keyword'][i])
                c+=1
            self.train['keyword'] = f
            self.train = self.train.drop(['location'],axis = 1)
            return self.train
            
                    
            

            

In [231]:
p = preprocess(df,train)
newtrain = p.transform()

v = preprocess(df,validation,validation = True)
newval = v.transform()

y = preprocess(df,dftest,test = True)
newtest = y.transform()

C:\Users\HP\AppData\Local\Temp/ipykernel_3432/434864315.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.train['text'] = u
C:\Users\HP\AppData\Local\Temp/ipykernel_3432/434864315.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.train['keyword'] = f


In [232]:
newtrain

,id,keyword,text,target
0,1,earthquake,our deeds are the reason of this earthquake ma...,1
1,4,forest,forest fire near la ronge sask canada,1
2,5,evacuation,all residents asked to shelter in place are be...,1
3,6,evacuation,13 000 people receive wildfires evacuation ord...,1
4,7,smoke,just got sent this photo from ruby alaska as s...,1
...,...,...,...,...
6994,10030,twister,bulgarian tittie twister by noemotion produced...,0
6995,10031,twister,7 speed nexus shifter 9 for community cargo bike,0
6996,10032,twister,anyone wanna come over and watch twister with ...,0
6998,10035,twister,twister with bill paxton and helen hunt,0


In [233]:
newval

,id,keyword,text,target
7000,10037,twister,happy 24 twister thank you for all the laughs ...,0
7001,10038,twister,how many men would a human hew if a human coul...,0
7002,10039,twister,i liked a video extreme paint twister,0
7003,10040,twister,tornadoes el nino may give canada s twister se...,1
7004,10041,twister,like gg was bad in the end but at least i care...,0
...,...,...,...,...
7608,10869,bridge,two giant cranes holding a bridge collapse int...,1
7609,10870,wild,ahrary the out of control wild fires in califo...,1
7610,10871,volcano,m1 94 01 04 utc 5km s of volcano hawaii,1
7611,10872,police,police investigating after an e bike collided ...,1


In [234]:
newtest

,id,keyword,text
0,0,crash,just happened a terrible car crash
1,2,earthquake,heard about earthquake is different cities sta...
2,3,forest,there is a forest fire at spot pond geese are ...
3,9,apocalypse,apocalypse lighting spokane wildfires
4,11,typhoon,typhoon soudelor kills 28 in china and taiwan
...,...,...,...
3258,10861,earthquake,earthquake safety los angeles safety fasteners...
3259,10865,storm,storm in ri worse than last hurricane my city ...
3260,10868,derailment,green line derailment in chicago
3261,10874,hazardous,meg issues hazardous weather outlook hwo
